# Prepare Background (inactive) DNS Tunneling Attack Dataset

## Overview:

This notebook will focus on creating a background (inactive) DNS Tunneling attack dataset based on a small sample of data collected by performing real DNS Tunneling attacks in a controlled environment.<br>
The dataset that this notebook creates closely represents real-world data and was used to train our SVM model.<br>  
It is worth noteing that the sample dataset we collected does not contain any missing values or any outliers due to the fact we tested each part of the collection process and verified that it is correct.<br>
In this notebook we have generated an attack dataset with 50,000 flows of the DNS Tunneling attack based on the samples we collected when running a DNS Tunneling attacks in various configurations using the well known DNScat2 tool when the victim host has a DNS tunnel open <u>but no</u> commands or data are being transmitted through it, the tunnel is open but not actively used by the attacker.<br>

## Imports & Global Variables:

In [45]:
import pandas as pd
import numpy as np
import random

NUM_OF_ROWS = 50000
ATTACK_NAME = 'DNS'

In [46]:
# the following command will make it so that when we print the dataframe we will see all the columns
pd.set_option('display.max_columns', None)

---

## Load the sample dataset:

In [47]:
# import the attack sample dataset
dns_samples = pd.read_csv('dns_samples_background.csv')
print(f'Dataset Shape: {dns_samples.shape}')
dns_samples

Dataset Shape: (10, 27)


,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
0,0,0,16,36,36,44,36.461538,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.272727,101,158,22,24,4908,2596,88,22.814415,1.096587,0.262235,0.461227
1,0,0,48,48,36,66,38.000000,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.318182,101,158,33,35,7368,3894,132,34.926907,1.105063,0.266618,0.465087
2,0,0,48,36,52,68,38.571429,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.823529,101,158,34,36,7660,4012,136,35.418652,1.105114,0.262360,0.456099
3,0,0,48,52,34,68,37.840000,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.626866,101,158,33,35,7580,3894,134,35.846157,1.102855,0.269520,0.464188
4,0,0,44,56,36,68,37.520000,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.117647,101,158,34,36,7564,4012,136,35.797906,1.107464,0.265170,0.461400
5,0,0,28,40,68,68,37.294118,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.823529,101,158,34,36,7660,4012,136,35.301731,1.102360,0.261494,0.454823
6,0,0,52,40,44,68,38.521739,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.647059,101,158,34,36,7636,4012,136,35.763095,1.096167,0.264912,0.461058
7,0,0,40,48,48,68,37.636364,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.441176,101,158,34,36,7608,4012,136,35.993821,1.098705,0.266621,0.464297
8,0,0,44,48,40,66,37.826087,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.348485,101,158,33,35,7372,3894,132,34.958758,1.098239,0.266861,0.465344
9,0,0,52,32,48,66,38.952381,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.893939,101,158,33,35,7444,3894,132,34.897108,1.098657,0.266390,0.463545


### Find the columns that we need to synthesis data for:

In [48]:
columns_to_gather = dns_samples.replace(0, np.nan) #replace all 0 values with null
columns_to_gather = columns_to_gather.dropna(how = 'all', axis = 1).columns.tolist() #remove all columns where there are null values
columns_to_gather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['CName Record Count',
 'TXT Record Count',
 'MX Record Count',
 'DF Flag Count',
 'Average Response Data Length',
 'Min Response Data Length',
 'Max Response Data Length',
 'Average Domain Name Length',
 'Min Domain Name Length',
 'Max Domain Name Length',
 'Average Sub Domain Name Length',
 'Min Sub Domain Name Length',
 'Max Sub Domain Name Length',
 'Average Packet Length',
 'Min Packet Length',
 'Max Packet Length',
 'Number of Domian Names',
 'Number of Sub Domian Names',
 'Total Length of Fwd Packet',
 'Total Length of Bwd Packet',
 'Total Number of Packets',
 'Flow Duration',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

### Find an approximate minimum and maximum values of each column:

In [49]:
# find the minimum and maximum values for each column, scale the range (reduce min by 10% and increase max by 35%), and store the results in a dictionary.
min_max_dict = {col: (float(dns_samples[col].min() * 0.9), float(dns_samples[col].max() * 1.35)) for col in columns_to_gather}

# print the min max dictionary
for col, (min_val, max_val) in min_max_dict.items():
    print(f'{col:<30} | Min: {min_val:.2f} | Max: {max_val:.2f}')

CName Record Count             | Min: 14.40 | Max: 70.20
TXT Record Count               | Min: 28.80 | Max: 75.60
MX Record Count                | Min: 30.60 | Max: 91.80
DF Flag Count                  | Min: 39.60 | Max: 91.80
Average Response Data Length   | Min: 32.82 | Max: 52.59
Min Response Data Length       | Min: 30.60 | Max: 45.90
Max Response Data Length       | Min: 37.80 | Max: 56.70
Average Domain Name Length     | Min: 37.80 | Max: 56.70
Min Domain Name Length         | Min: 37.80 | Max: 56.70
Max Domain Name Length         | Min: 37.80 | Max: 56.70
Average Sub Domain Name Length | Min: 12.90 | Max: 19.35
Min Sub Domain Name Length     | Min: 12.90 | Max: 19.35
Max Sub Domain Name Length     | Min: 12.90 | Max: 19.35
Average Packet Length          | Min: 114.41 | Max: 172.66
Min Packet Length              | Min: 90.90 | Max: 136.35
Max Packet Length              | Min: 142.20 | Max: 213.30
Number of Domian Names         | Min: 19.80 | Max: 45.90
Number of Sub Domian Names

### Create the base attack dataset (full of zeros):

In [50]:
# creating an empty dataframe before adding values to it
dns_dataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dns_samples.columns))), columns = dns_samples.columns)
dns_dataset.head(3)

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Find the columns with constant zero values based on samples:

In [51]:
# adding zeros to all columns that should not have any values
zero_columns = [col for col in dns_samples.columns if col not in columns_to_gather]
for col in zero_columns:
    dns_dataset[col] = int(0)
zero_columns

['A Record Count', 'AAAA Record Count']

---

## Filling in values based on collected samples:

### Firstly we insert data into columns that are not related to each other:

In [52]:
constant_columns = ['TXT Record Count', 'MX Record Count', 'CName Record Count', 'DF Flag Count', 'Number of Sub Domian Names', 'Total Number of Packets']

# adding the attack feature values to the dataset at random based on the smaple data using the minimum maximum dict
for col in constant_columns:
    dns_dataset[col] = np.random.randint(min_max_dict[col][0], min_max_dict[col][1], NUM_OF_ROWS)

## Then we fill values into columns that have a certain correlation between them:

A correlation between two or more columns is common in our dataset since most features are inherently related. All of them are derived from network packet traffic.<br>
For example, as the **total number of packets** increases, the **total length of fwd packet** and **total length of bwd packet** are both likely to increases as well. This happens because packets can be either forward or backward, and the more packets there are, the more fwd and bwd traffic (packets) there will be.<br>

### Correlation between multiple columns based on the values in the list_of_correlation:

In [53]:
lists_of_correlation = [
    ['Number of Sub Domian Names', 'Number of Domian Names'],
    ['Average Response Data Length', 'Min Response Data Length'],
    ['Total Number of Packets', 'Total Length of Fwd Packet', 'Total Length of Bwd Packet'],
    ['Average Packet Length', 'Min Packet Length', 'Max Packet Length']
]

In [ ]:
# for each list of correlation, calculate the correlation between the first column in the list and the rest of the columns
for l in lists_of_correlation:
    dns_dataset[l[0]] = np.random.randint(min_max_dict[l[0]][0], min_max_dict[l[0]][1], NUM_OF_ROWS)
    if l[0] == 'Average Response Data Length':
        dns_dataset[l[0]] = np.random.uniform(min_max_dict[l[0]][0], min_max_dict[l[0]][1], NUM_OF_ROWS)

    # finding the correlation between the first column to the rest of the columns in order to create new data
    independent_col = dns_samples[l[0]].values.reshape(-1, 1) 
    dependent_cols = dns_samples[l[1:]].values

    # using least squares regression to find scaling factors that best approximate the relationship between the columns
    scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]
    scaling_factors = [(name, float(factor)) for name, factor in zip(l[1:], scaling_factors.flatten())]
    print(l[0]+':')
    for val in scaling_factors:
        print(val)
    print('-'*50)

    # precompute values using random deltas for all rows and all dependent columns
    for col, factor in scaling_factors:
        deltas = np.random.uniform(factor * 0.1, factor * 0.2, size = NUM_OF_ROWS)
        signs = np.random.choice([-1, 1], size = NUM_OF_ROWS)
        updated_factors = factor + signs * deltas

        # insert the data into the attack dataset for each column
        dns_dataset[col] = dns_dataset[l[0]] * updated_factors

Number of Sub Domian Names:
('Number of Domian Names', 0.9424556707929075)
--------------------------------------------------
Average Response Data Length:
('Min Response Data Length', 0.897701272076271)
--------------------------------------------------
Total Number of Packets:
('Total Length of Fwd Packet', 56.09391500246403)
('Total Length of Bwd Packet', 29.45361744068712)
--------------------------------------------------
Average Packet Length:
('Min Packet Length', 0.7919591927438456)
('Max Packet Length', 1.238906459935917)
--------------------------------------------------


In [55]:
dns_dataset

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
0,0,0,45,64,66,73,35.055542,25.604641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150,130.734636,207.805658,25.660828,34,8251.461564,4309.270309,123,0.0,0.0,0.0,0.0
1,0,0,36,41,75,84,37.344191,37.743219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,152,104.051622,152.683298,31.397735,29,9058.004332,4672.383572,135,0.0,0.0,0.0,0.0
2,0,0,33,48,80,84,44.329421,47.211909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125,114.334082,170.596098,28.481453,35,6712.454876,3623.616300,105,0.0,0.0,0.0,0.0
3,0,0,35,52,86,46,33.239875,24.960334,0.0,0.0,0.0,0.0,0.0,0.0,0.0,171,150.812359,181.359123,24.236000,31,6904.650457,4047.686943,153,0.0,0.0,0.0,0.0
4,0,0,60,41,34,76,44.794844,47.871613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,145,95.873444,145.995543,34.929070,33,5890.096573,2408.944795,91,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,63,59,81,41,44.860077,33.342025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,165,109.894449,166.426857,37.782910,46,5910.011059,2449.892805,95,0.0,0.0,0.0,0.0
49996,0,0,51,34,34,85,33.528218,35.827022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118,80.369747,121.704487,36.121267,43,11131.639358,4246.256837,171,0.0,0.0,0.0,0.0
49997,0,0,66,42,79,83,46.122599,33.840572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143,97.192030,210.747994,48.864349,46,7465.040079,5515.374185,163,0.0,0.0,0.0,0.0
49998,0,0,51,32,71,72,45.311973,45.921849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,147,98.011467,150.903525,52.865987,47,10225.198145,5594.120794,160,0.0,0.0,0.0,0.0


### Then we insert data into columns that have the same values:

In [56]:
# this is a list of list, each inner list holds columns that should have the same values
same_values = [['Max Response Data Length', 'Average Domain Name Length', 'Min Domain Name Length', 'Max Domain Name Length'], 
              ['Average Sub Domain Name Length', 'Min Sub Domain Name Length', 'Max Sub Domain Name Length']]

# for each list of columns, generate a vector with values based on the minimum and maximum dict and insert that vector to each column in the current list
for l in same_values:
    val = np.random.uniform(min_max_dict[l[0]][0], min_max_dict[l[0]][1], NUM_OF_ROWS)
    for col in l:
        dns_dataset[col] = val

dns_dataset.head(10)

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
0,0,0,45,64,66,73,35.055542,25.604641,48.773102,48.773102,48.773102,48.773102,18.950731,18.950731,18.950731,150,130.734636,207.805658,25.660828,34,8251.461564,4309.270309,123,0.0,0.0,0.0,0.0
1,0,0,36,41,75,84,37.344191,37.743219,41.770877,41.770877,41.770877,41.770877,12.956591,12.956591,12.956591,152,104.051622,152.683298,31.397735,29,9058.004332,4672.383572,135,0.0,0.0,0.0,0.0
2,0,0,33,48,80,84,44.329421,47.211909,40.508424,40.508424,40.508424,40.508424,18.881063,18.881063,18.881063,125,114.334082,170.596098,28.481453,35,6712.454876,3623.616300,105,0.0,0.0,0.0,0.0
3,0,0,35,52,86,46,33.239875,24.960334,47.718353,47.718353,47.718353,47.718353,15.500654,15.500654,15.500654,171,150.812359,181.359123,24.236000,31,6904.650457,4047.686943,153,0.0,0.0,0.0,0.0
4,0,0,60,41,34,76,44.794844,47.871613,54.313867,54.313867,54.313867,54.313867,18.711703,18.711703,18.711703,145,95.873444,145.995543,34.929070,33,5890.096573,2408.944795,91,0.0,0.0,0.0,0.0
5,0,0,58,51,89,46,34.053967,34.174871,43.895635,43.895635,43.895635,43.895635,18.739366,18.739366,18.739366,128,90.010706,133.537654,16.369836,21,7141.530770,2811.377920,115,0.0,0.0,0.0,0.0
6,0,0,67,44,70,55,37.964467,29.346612,50.373495,50.373495,50.373495,50.373495,14.200692,14.200692,14.200692,140,89.842681,150.230862,18.512697,22,7661.433751,3977.439598,121,0.0,0.0,0.0,0.0
7,0,0,63,56,90,58,34.105373,25.182676,52.414530,52.414530,52.414530,52.414530,13.461289,13.461289,13.461289,143,128.353834,153.104817,17.789092,22,6568.573977,3332.518729,102,0.0,0.0,0.0,0.0
8,0,0,42,74,54,75,50.781999,38.140629,41.364367,41.364367,41.364367,41.364367,15.759061,15.759061,15.759061,129,120.358010,182.204824,37.677895,35,7520.858885,4047.289361,121,0.0,0.0,0.0,0.0
9,0,0,18,61,47,72,36.674720,36.633756,39.287253,39.287253,39.287253,39.287253,14.050964,14.050964,14.050964,151,103.654337,220.276027,25.244757,32,5338.915271,2009.820076,83,0.0,0.0,0.0,0.0


### Correlation between 'IAT Mean' and all of the following: 'IAT Max', 'IAT Std', 'Flow Duration':

In [57]:
# finding the correlation between the 'IAT Mean' column to the rest of the columns in order to create new data
correlation = ['IAT Mean', 'IAT Max', 'IAT Std', 'Flow Duration']
independent_col = dns_samples[correlation[0]].values.reshape(-1, 1) #column 'IAT Mean'
dependent_cols = dns_samples[correlation[1:]].values 

# using least squares regression to find scaling factors that best approximate the relationship between 'IAT Mean' and the rest of the columns in correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, float(factor)) for name, factor in zip(correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('IAT Max', 4.151419427246799)
('IAT Std', 1.7408151217079082)
('Flow Duration', 128.88859283302108)


In [58]:
dns_dataset['IAT Mean'] = np.random.uniform(min_max_dict['IAT Mean'][0], min_max_dict['IAT Mean'][1], NUM_OF_ROWS)

# iterating over all rows we need to add values
for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'IAT Mean'
    delta = random.uniform(factor * 0.1, factor * 0.2) 
    updated_factor = factor + delta
    dns_dataset[col] = dns_dataset['IAT Mean'] * updated_factor


---

## Adding the Label column:

In [59]:
# adding a label to the dataset
dns_dataset['Label'] = ATTACK_NAME

---

## Validate that the generated data looks valid by comparing the samples with the generated dataset:

Make sure that the data that needs to be of type Integer will be Integer for consistency.  

In [60]:
int_columns = ['Min Response Data Length', 'Max Response Data Length', 'Average Domain Name Length', 'Min Domain Name Length',
                'Max Domain Name Length', 'Min Packet Length', 'Max Packet Length', 'Number of Domian Names', 'Number of Sub Domian Names',
                'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 'Total Number of Packets']

for col in int_columns:
    dns_dataset[col] = dns_dataset[col].astype(int)

In [61]:
dns_samples

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
0,0,0,16,36,36,44,36.461538,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.272727,101,158,22,24,4908,2596,88,22.814415,1.096587,0.262235,0.461227
1,0,0,48,48,36,66,38.000000,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.318182,101,158,33,35,7368,3894,132,34.926907,1.105063,0.266618,0.465087
2,0,0,48,36,52,68,38.571429,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.823529,101,158,34,36,7660,4012,136,35.418652,1.105114,0.262360,0.456099
3,0,0,48,52,34,68,37.840000,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.626866,101,158,33,35,7580,3894,134,35.846157,1.102855,0.269520,0.464188
4,0,0,44,56,36,68,37.520000,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.117647,101,158,34,36,7564,4012,136,35.797906,1.107464,0.265170,0.461400
5,0,0,28,40,68,68,37.294118,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.823529,101,158,34,36,7660,4012,136,35.301731,1.102360,0.261494,0.454823
6,0,0,52,40,44,68,38.521739,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.647059,101,158,34,36,7636,4012,136,35.763095,1.096167,0.264912,0.461058
7,0,0,40,48,48,68,37.636364,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.441176,101,158,34,36,7608,4012,136,35.993821,1.098705,0.266621,0.464297
8,0,0,44,48,40,66,37.826087,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.348485,101,158,33,35,7372,3894,132,34.958758,1.098239,0.266861,0.465344
9,0,0,52,32,48,66,38.952381,34,42,42.0,42,42,14.333333,14.333333,14.333333,127.893939,101,158,33,35,7444,3894,132,34.897108,1.098657,0.266390,0.463545


In [62]:
dns_dataset

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std,Label
0,0,0,45,64,66,73,35.055542,25,48,48,48,48,18.950731,18.950731,18.950731,150,130,207,25,34,8251,4309,123,47.008595,1.525348,0.326541,0.656521,DNS
1,0,0,36,41,75,84,37.344191,37,41,41,41,41,12.956591,12.956591,12.956591,152,104,152,31,29,9058,4672,135,47.489327,1.540947,0.329880,0.663235,DNS
2,0,0,33,48,80,84,44.329421,47,40,40,40,40,18.881063,18.881063,18.881063,125,114,170,28,35,6712,3623,105,37.383716,1.213037,0.259683,0.522100,DNS
3,0,0,35,52,86,46,33.239875,24,47,47,47,47,15.500654,15.500654,15.500654,171,150,181,24,31,6904,4047,153,45.685498,1.482416,0.317350,0.638043,DNS
4,0,0,60,41,34,76,44.794844,47,54,54,54,54,18.711703,18.711703,18.711703,145,95,145,34,33,5890,2408,91,50.157065,1.627510,0.348411,0.700493,DNS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,63,59,81,41,44.860077,33,47,47,47,47,14.895553,14.895553,14.895553,165,109,166,37,46,5910,2449,95,39.073894,1.267880,0.271423,0.545705,DNS
49996,0,0,51,34,34,85,33.528218,35,52,52,52,52,13.084380,13.084380,13.084380,118,80,121,36,43,11131,4246,171,43.805031,1.421398,0.304288,0.611780,DNS
49997,0,0,66,42,79,83,46.122599,33,40,40,40,40,15.052677,15.052677,15.052677,143,97,210,48,46,7465,5515,163,42.949647,1.393642,0.298346,0.599834,DNS
49998,0,0,51,32,71,72,45.311973,45,52,52,52,52,16.028160,16.028160,16.028160,147,98,150,52,47,10225,5594,160,50.945062,1.653079,0.353885,0.711498,DNS


In [63]:
dns_samples.describe()

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
count,10.0,10.0,10.000000,10.00000,10.000000,10.000000,10.000000,10.0,10.0,10.0,10.0,10.0,1.000000e+01,10.000000,10.000000,10.000000,10.0,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.0,0.0,42.000000,43.60000,44.200000,65.000000,37.862366,34.0,42.0,42.0,42.0,42.0,1.433333e+01,14.333333,14.333333,127.531314,101.0,158.0,32.400000,34.400000,7280.000000,3823.200000,129.800000,34.171855,1.101121,0.265218,0.461707
std,0.0,0.0,11.508451,7.87683,10.432854,7.438638,0.714916,0.0,0.0,0.0,0.0,0.0,3.601719e-15,0.000000,0.000000,0.268694,0.0,0.0,3.687818,3.687818,840.850363,435.162498,14.800901,4.011606,0.003966,0.002528,0.003659
min,0.0,0.0,16.000000,32.00000,34.000000,44.000000,36.461538,34.0,42.0,42.0,42.0,42.0,1.433333e+01,14.333333,14.333333,127.117647,101.0,158.0,22.000000,24.000000,4908.000000,2596.000000,88.000000,22.814415,1.096167,0.261494,0.454823
25%,0.0,0.0,41.000000,37.00000,36.000000,66.000000,37.549091,34.0,42.0,42.0,42.0,42.0,1.433333e+01,14.333333,14.333333,127.325758,101.0,158.0,33.000000,35.000000,7390.000000,3894.000000,132.000000,34.934870,1.098343,0.262998,0.461100
50%,0.0,0.0,46.000000,44.00000,42.000000,68.000000,37.833043,34.0,42.0,42.0,42.0,42.0,1.433333e+01,14.333333,14.333333,127.534021,101.0,158.0,33.500000,35.500000,7572.000000,3953.000000,135.000000,35.360191,1.100532,0.265780,0.462472
75%,0.0,0.0,48.000000,48.00000,48.000000,68.000000,38.391304,34.0,42.0,42.0,42.0,42.0,1.433333e+01,14.333333,14.333333,127.779412,101.0,158.0,34.000000,36.000000,7629.000000,4012.000000,136.000000,35.789203,1.104511,0.266620,0.464270
max,0.0,0.0,52.000000,56.00000,68.000000,68.000000,38.952381,34.0,42.0,42.0,42.0,42.0,1.433333e+01,14.333333,14.333333,127.893939,101.0,158.0,34.000000,36.000000,7660.000000,4012.000000,136.000000,35.993821,1.107464,0.269520,0.465344


In [64]:
dns_dataset.describe()

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
count,50000.0,50000.0,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.00000,50000.00000,50000.00000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,0.0,0.0,41.375920,50.964540,60.010100,64.597280,42.668408,37.823520,46.73658,46.73658,46.73658,46.73658,16.125347,16.125347,16.125347,142.44912,112.177280,175.784420,31.514300,33.993900,7331.177600,3844.193340,130.511460,43.126444,1.399379,0.299574,0.602303
std,0.0,0.0,16.091201,13.553551,17.624327,15.000951,5.703743,7.838758,5.44630,5.44630,5.44630,5.44630,1.861219,1.861219,1.861219,16.77843,21.841593,34.313421,8.875222,7.772379,2049.231666,1072.464746,30.088957,5.351099,0.173634,0.037171,0.074733
min,0.0,0.0,14.000000,28.000000,30.000000,39.000000,32.815535,23.000000,37.00000,37.00000,37.00000,37.00000,12.900282,12.900282,12.900282,114.00000,72.000000,112.000000,15.000000,21.000000,3547.000000,1861.000000,79.000000,33.880388,1.099360,0.235347,0.473173
25%,0.0,0.0,27.000000,39.000000,45.000000,52.000000,37.759459,32.000000,42.00000,42.00000,42.00000,42.00000,14.517511,14.517511,14.517511,128.00000,95.000000,149.000000,25.000000,27.000000,5720.000000,3000.000000,104.000000,38.459971,1.247960,0.267159,0.537131
50%,0.0,0.0,41.000000,51.000000,60.000000,65.000000,42.647196,37.000000,47.00000,47.00000,47.00000,47.00000,16.138877,16.138877,16.138877,142.00000,110.000000,171.000000,31.000000,34.000000,7167.000000,3756.000000,131.000000,43.123206,1.399274,0.299551,0.602258
75%,0.0,0.0,55.000000,63.000000,75.000000,78.000000,47.573646,43.000000,51.00000,51.00000,51.00000,51.00000,17.735989,17.735989,17.735989,157.00000,129.000000,202.000000,37.000000,41.000000,8666.000000,4538.000000,157.000000,47.766997,1.549957,0.331809,0.667113
max,0.0,0.0,69.000000,74.000000,90.000000,90.000000,52.585404,56.000000,56.00000,56.00000,56.00000,56.00000,19.349860,19.349860,19.349860,171.00000,162.000000,254.000000,53.000000,47.000000,12249.000000,6429.000000,182.000000,52.379689,1.699631,0.363851,0.731534


---

## At the end we save the dataset as a CSV file

In [65]:
print(f'Attack Dataset Shape: {dns_dataset.shape}')

Attack Dataset Shape: (50000, 28)


In [66]:
# save the dataset
dns_dataset.to_csv('dns_background_dataset.csv', index=False)